In [ ]:
#general packages
import pandas as pd
import numpy as np
from collections import Counter
import tifffile as tf
from skimage.measure import regionprops
#plotting packages
import matplotlib.pyplot as plt
import seaborn as sns
#custom function
from post_analysis import *
%config InlineBackend.figure_format='retina'

In [ ]:
#read in final mtx
#grab all channels
all_channels = []

for c in [1,2,3]:
    mtx = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/06122022_4kgenes/notebook_pyfiles/genebycell/final_222_33_heg_svm_0p15_diff1_fdr10/final/genebycell_{c}.csv", index_col=0)
    #mtx = mtx.T
    #mtx = mtx[mtx.sum(axis=1)>100].T
    all_channels.append(mtx)

mtx = pd.concat(all_channels, axis=0).fillna(0)
mtx = mtx.T
mtx = mtx[mtx.sum(axis=1)>300].T

In [ ]:
#take a look
mtx

In [ ]:
#read in rnaseq data
rnaseq = pd.read_csv("./RNAseq_files/NIH3T3_CCS_TPM_REP1.csv")
rnaseq.columns = ["Genes","TPM"]

In [ ]:
#codebook
codebooks = ["codebook_string_647.csv","codebook_string_561.csv","codebook_string_488.csv"]
codebook = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/06122022_4kgenes/notebook_pyfiles/decoding_files/SVM_Feature_Radial_Decoding/codebook_converter/{codebooks[c-1]}", index_col=0)
#separate into true and false codebook
fakebook = codebook[codebook.index.str.startswith("fake")]
codebook = codebook.drop(fakebook.index)

In [ ]:
#calculate fdr
fp, fake = percent_false_positive(mtx, codebook, fakebook)
percent_fp = fp["FP raw"].mean()
mean_counts = fp["total_real"].mean()
sum_counts = fp["total_counts"].sum()
norm_fpr = fp["FDR"].mean()
fp_list = [percent_fp,norm_fpr,mean_counts,sum_counts]

In [ ]:
#take a look at fdr results
df_stats = pd.DataFrame(fp_list).T
df_stats.columns = ["percent fp","false positive rate","mean true counts", "total sum"]
df_stats

In [ ]:
#convert data to pseudobulk rnaseq data
bulk = pd.DataFrame(mtx.mean(axis=1)).reset_index()
bulk.columns = ["Genes", "Counts"]
bulk["Genes"] = bulk["Genes"].str.lower()
rnaseq["Genes"] = rnaseq["Genes"].str.lower()
#merge
comb_1 = pd.merge(rnaseq,bulk)
#pearson's correlation
r = pearsonr(comb_1["TPM"],comb_1["Counts"])
r = round(r[0],2)

In [ ]:
#get log2 + 1
comb_1["Log Counts"] = np.log10(comb_1["Counts"]+0.1)
comb_1["Log TPM"] = np.log10(comb_1["TPM"]+0.1)

In [ ]:
#RNA-seq plot
sns.set_style("white")
joint_kws=dict(gridsize=50)
hexplot = sns.jointplot(data=comb_1, x="Log TPM", y="Log Counts", kind="hex",mincnt=0.1, 
              cmap="plasma", dropna=True, joint_kws=joint_kws)
plt.xlabel("Bulk RNAseq Log10(TPM+0.1)", fontsize=12)
plt.ylabel("Pseudobulk Log10(Counts+0.1)", fontsize=12)
hexplot.ax_marg_x.remove()
hexplot.ax_marg_y.remove()
plt.annotate(f"Pearson's r= {r}", (-1.0,1.6), fontsize=12)
plt.title("All Channels", fontweight="bold")
plt.colorbar()
sns.despine()
plt.show()

In [ ]:
#read in smfish 
#smfish = pd.read_csv("./nih3t3_smfish/smFISH_results.csv").T
smfish = pd.read_csv("./nih3t3_smfish/smfish_27gene_custom_thresh_2.csv", index_col=0)
_150genes = pd.read_csv("./150genes_diff0.csv", index_col=0)

In [ ]:
correlation(mtx,smfish, label_x="smFISH", label_y="LANTERN",
            title="All Channels", cell_size_normalized=False, 
            return_comb_df=False, log=False)

In [ ]:
correlation(mtx,smfish, label_x="smFISH", label_y="LANTERN",
            title="All Channels", cell_size_normalized=False, 
            return_comb_df=False, log=True)

In [ ]:
correlation(mtx,_150genes, label_x="smFISH", label_y="LANTERN",
            title="All Channels", cell_size_normalized=False, 
            return_comb_df=False, log=False)

In [ ]:
correlation(mtx,_150genes, label_x="smFISH", label_y="LANTERN",
            title="All Channels", cell_size_normalized=False, 
            return_comb_df=False, log=True)

# Percent decoded

In [ ]:
#get average percent decoded
percent_decoded_list = []
for i in range(52):
    for z in range(1):
        try:
            src = f"/groups/CaiLab/personal/Lex/raw/06122022_4kgenes/notebook_pyfiles/decoded/final_222_33_heg_svm_0p15_diff1_fdr10/Channel_3/Pos_{i}/percent_decoded_z_{z}.txt"
            with open(src) as f:
                decoded = f.readlines()[0].split(" ")[-1]
                f.close()
                percent_decoded_list.append(float(decoded))
        except FileNotFoundError:
            continue

In [ ]:
np.mean(percent_decoded_list)

# Random checks

In [ ]:
def calc_density(locations, mask, pos=0, counts_threshold = 100, density = "gene", pixel=0.1):
    
    """Function to return gene or spot density"""
    if density == "spot":
        area_per_cell = []
        info = regionprops(mask)
        for cell in info:
            area = cell.area
            label = cell.label
            area_per_cell.append([label,area])

        df = pd.DataFrame(area_per_cell)
        df.columns = ["cell number","area"]

        df_loc = pd.DataFrame(locations.groupby("cell number").size()).reset_index()
        df_loc.columns = ["cell number", "counts"]

        df_final = pd.merge(df_loc,df)

        gene_density = (df_final.counts/df_final.area).values
        
    else:
        area_per_cell = []
        info = regionprops(mask)
        for cell in info:
            area = cell.area
            label = cell.label
            area_per_cell.append([label,area])

        df = pd.DataFrame(area_per_cell)
        df.columns = ["cell number","area"]
        
        gene_density = pd.DataFrame()
        for cell in locations["cell number"].unique():
            gene_counts = locations[locations["cell number"]==cell].groupby("genes").size()
            if gene_counts.sum() < counts_threshold:
                continue
            gene_counts = gene_counts / df[df["cell number"] == cell].area.iloc[0]
            gene_counts = gene_counts/(pixel**2)
            final_counts = pd.DataFrame(gene_counts)
            gene_density[f"cell{cell}_pos{pos}"] = gene_counts
        gene_density = gene_density.fillna(0)
        
    
    return gene_density

In [ ]:
channel = 3
pixel = 0.1
current_df = []
ref_df = []
for i in range(74):
    try:
        current = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/06122022_4kgenes/notebook_pyfiles/decoded/final_222_33_heg_svm_0p15_diff1_fdr10/Channel_{channel}/Pos_{i}/diff_1_minseed_3_z_0_finalgenes.csv")
        #get cell masks
        mask = tf.imread(f"/groups/CaiLab/personal/Lex/raw/06122022_4kgenes/notebook_pyfiles/edges_deleted/MMStack_Pos{i}_z0.tif")
        current_den = calc_density(current, mask, pos=i, counts_threshold = 100, density = "gene", pixel=pixel)
        current_df.append(current_den)
    except:
        continue
        
current_df = pd.concat(current_df,axis=1).fillna(0)

for i in range(64):
    try:
        ref = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/050222_150genes4binding/notebook_pyfiles/decoded/final_11p52_33_heg_svm_0p15_diff0_newdecoder2/Channel_1/Pos_{i}/diff_0_minseed_3_z_0_finalgenes.csv")
        mask = tf.imread(f"/groups/CaiLab/personal/Lex/raw/050222_150genes4binding/notebook_pyfiles/edges_deleted/MMStack_Pos{i}.tif")
        ref_den = calc_density(ref, mask, pos=i, counts_threshold = 100, density = "gene", pixel=pixel)
        ref_df.append(ref_den)
    except:
        continue
        
ref_df = pd.concat(ref_df,axis=1).fillna(0)

In [ ]:
correlation(current_df,ref_df, label_x="150 gene LANTERN", label_y="LANTERN",
            title="All Channels", cell_size_normalized=True, 
            return_comb_df=False, log=False)

In [ ]:
channel = 3
pixel = 0.1
current_df = []
for i in range(74):
    try:
        current = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/06122022_4kgenes/notebook_pyfiles/decoded/final_222_33_heg_svm_0p15_diff1_fdr10/Channel_{channel}/Pos_{i}/diff_1_minseed_3_z_0_finalgenes.csv")
        #get cell masks
        mask = tf.imread(f"/groups/CaiLab/personal/Lex/raw/06122022_4kgenes/notebook_pyfiles/edges_deleted/MMStack_Pos{i}_z0.tif")
        current_den = calc_density(current, mask, pos=i, counts_threshold = 100, density = "gene", pixel=pixel)
        current_df.append(current_den)
    except:
        continue
        
#alter smfish genebycell
genebycell = pd.read_csv("./nih3t3_smfish/smfish_27gene_custom_thresh_2.csv", index_col=0)

area_smfish = []
for i in range(31):
    mask = tf.imread(f"/groups/CaiLab/personal/Lex/raw/042022_40genes_smfish/notebook_pyfiles/edges_deleted/MMStack_Pos{i}.tif")
    area_per_cell = []
    info = regionprops(mask)
    for cell in info:
        area = cell.area
        label = cell.label
        area_per_cell.append([label,area])

    df = pd.DataFrame(area_per_cell)
    df.columns = ["cell number","area"]
    df["pos"]=i
    area_smfish.append(df)
area_smfish = pd.concat(area_smfish)

#get density for each column
for cell_id in genebycell.columns:
    cell_num = int(cell_id.split("_")[0].replace("cell",""))
    cell_pos = int(cell_id.split("_")[1].replace("pos",""))
    get_area = area_smfish[(area_smfish["cell number"] == cell_num)&(area_smfish["pos"] == cell_pos)]
    area = get_area.area.iloc[0]
    genebycell[cell_id]=(genebycell[cell_id]/area)/(pixel**2)

In [ ]:
mtx = pd.concat(current_df,axis=1).fillna(0)
#mtx_ref = pd.concat(old_df,axis=1).fillna(0)
mtx_ref = genebycell

In [ ]:
correlation(mtx,mtx_ref, label_x="smFISH", label_y="LANTERN",
            title="4000 gene experiment (061222) Channel 488 nm", return_comb_df=False)

In [ ]:
#compare undefineds
percent_und_list = []
channel=3
for i in range(25):
    current_und = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/06122022_4kgenes/notebook_pyfiles/decoded/final_222_33_heg_svm_0p15_diff1_fdr10/Channel_{channel}/Pos_{i}/dots_used_undefined_z_0.csv")
    current_und = current_und[current_und["hyb"] < 48]
    current_loc = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/06122022_4kgenes/notebook_pyfiles/dots_detected/Channel_{channel}/genes_in_cells/Pos{i}/locations_z_0.csv")
    current_loc = current_loc[current_loc["hyb"] < 48]
    current_per = len(current_und)/len(current_loc)
    
    old_und = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/06122022_4kgenes/notebook_pyfiles/decoded/final_11p52_33_heg_svm_0p15_diff1_fdr10/Channel_{channel}/Pos_{i}/dots_used_undefined_z_0.csv")
    old_und = old_und[old_und["hyb"] <48]
    old_loc = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/06122022_4kgenes/notebook_pyfiles/dots_detected/Channel_{channel}/genes_in_cells/Pos{i}/locations_z_0.csv")
    old_loc = old_loc[old_loc["hyb"] <48]
    old_per = len(old_und)/len(old_loc)
    
    diff = current_per-old_per
    
    percent_und_list.append(diff)

In [ ]:
np.mean(percent_und_list)*100

In [ ]:
#compare fakes
percent_fake_list = []
channel=3
for i in range(25):
    current_fake = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/06122022_4kgenes/notebook_pyfiles/decoded/final_222_33_heg_svm_0p15_diff1_fdr10/Channel_{channel}/Pos_{i}/dots_used_fakes_z_0.csv")
    current_fake = current_fake[current_fake["hyb"] < 48]
    current_loc = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/06122022_4kgenes/notebook_pyfiles/dots_detected/Channel_{channel}/genes_in_cells/Pos{i}/locations_z_0.csv")
    current_loc = current_loc[current_loc["hyb"] < 48]
    current_per = len(current_fake)/len(current_loc)
    
    old_fake = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/06122022_4kgenes/notebook_pyfiles/decoded/final_11p52_33_heg_svm_0p15_diff1_fdr10/Channel_{channel}/Pos_{i}/dots_used_fakes_z_0.csv")
    old_fake = old_fake[old_fake["hyb"] <48]
    old_loc = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/06122022_4kgenes/notebook_pyfiles/dots_detected/Channel_{channel}/genes_in_cells/Pos{i}/locations_z_0.csv")
    old_loc = old_loc[old_loc["hyb"] <48]
    old_per = len(old_fake)/len(old_loc)
    
    diff = current_per-old_per
    
    percent_fake_list.append(diff)

In [ ]:
np.mean(percent_fake_list)

# Identify problematic hybs

In [ ]:
#collapse into gene counts
counts_df = Counter(fake["genes"])
#change to df
counts_df = pd.DataFrame.from_dict(counts_df, orient='index').reset_index()
counts_df

In [ ]:
#check codebook
codebook = pd.read_csv("/groups/CaiLab/personal/Lex/raw/150genes3bind_040622/barcode_key/codebook_647nm.csv", index_col=0)

In [ ]:
#lowercase gene names
codebook.index = codebook.index.str.lower()

In [ ]:
#get top 30
maj_fakes = counts_df.sort_values(0, ascending=False).head(30)

In [ ]:
#get troubled hybs
troubled_hybs = []
for fakes in maj_fakes["index"]:
    troubled_hybs.append(codebook.loc[fakes].values.tolist())

In [ ]:
#convert to array
hybstocheck = np.array(troubled_hybs)

In [ ]:
#take a look
hybstocheck

In [ ]:
#get mode for each round
bad_hybs = []
for i in range(len(hybstocheck[0])):
    vals,counts = np.unique(hybstocheck[:,i], return_counts=True)
    index = np.argmax(counts)
    bad_hyb = vals[index]
    bad_hybs.append(bad_hyb)

print(f"Following hybs by rounds are problematic:{bad_hybs}" )